<h1 style = "text-align: center; color: green; font-size: 32px"> <b> PreProt 🧬 </b> </h1>
<p style = "text-align: center; color: cyan; font-size: 18px"> Modelo de Machine Learning para predição de proteínas da <i> E. Coli </i> </p>
<p style = "text-align: center; font-size: 12px"> Projeto de conclusão para o curso de Data Science & analytics pela USP/ESALQ </p>
<p style = "text-align: center; font-size: 10px"> Desenvolvido por Fernando Falat, orientado por Miriam Martin </p>
<b style = "font-size: 24px" > O que é o PreProt? </b>
<p style = "font-size: 14px"> PreProt é um projeto de implementação de Machine Learning, mais especificamente Naive Bayes, para avaliar a eficácia e acurácia do modelo dentro do campo da bioinformática, através de testes de eficácia como o F test e outros parâmetros. Mais informações do projeto podem ser encontradas na <a href = "readme.txt"> documentação </a> e na <a href = "https://docs.google.com/document/d/1evF9-wIk1tZ6xFhq2hgq4nII7toi9ARP/edit)"> monografia </a>. </p>

<h3 style = "font-size: 24px"> <b> Formato de arquivos </b> </h3>
<p style = "font-size: 14px"> Na bioinformática, sequências de DNA e proteínas são os tipos de dados mais comuns e por padrão são armazenadas em um formato de arquivo "FASTA" (fast-all), "FAST-P" (proteínas) ou "FAST-N" (nucleotídeos).
<br></br>
<b> DNA: </b> A,T,C,G
<br></br>
<b> AMINOÁCIDOS: </b>Acrônimos de letras, ex: A para alanina </p>
<p style = "font-size: 14px"> Para abrir arquivos nesse formato, utiliza-se o SeqIO da biblioteca BioPython. Essa classe possibilta uma interface para trabalhar com esses tipos de dados. </p>

In [ ]:
# Importando biblioteca para leitura de arquivos FASTA
from Bio import SeqIO

# Abrindo o arquivo e apresentando algumas informações
sequences = [] # Criando uma lista vazia
for seq_record in SeqIO.parse("datasets/protein_sequences.fasta", "fasta"):
    # Adicionando o record na lista vazia
    sequences.append(str(seq_record.seq))
    # printando a sequência
    print(seq_record.seq)
    # printando o identificador da sequência (proteína)
    print(seq_record.id)
    # printando o tamanho da sequência (cadeia de aminoácidos)
    print(len(seq_record))
print("Número de sequências: ", len(sequences))

In [ ]:
# Sequencia de aminoácidos da E. Coli
print(sequences)

<style>
  p {
    text-align: justify;
    font-size: 14px !important;
  }
</style>

<h3 style = "font-size: 24px"> <b> Introdução </b> </h3>

<p style = "font-size: 14px"> O output fornecido acima representa a sequência de aminoácidos presentes no genoma da cepa de <i>E. Coli</i> em questão. Podemos ver que cada sequência de aminoácidos representa uma proteína com nome "ecmdb_XXXXXX".</p>
<div style = "font-size: 14px">
<p align = "justify" style = "font-size: 14px;"> A fim de que ocorra a formação de uma proteína, é necessário percorrer um conjunto de etapas. Iniciando-se pelos ácidos nucleicos, que constituem a base do DNA (ATCG):
<ol type = "1">
<li> <b> Transcrição: </b> O DNA é transicro em RNA mensageiro (mRNA) através da RNA polimerase.</li> 
<br></br>
<li> <b> Tradução: </b> O mRNA é lido pelos ribossomos, decodificando os códons (sequência de 3 ácidos nucleicos), então entram os tRNAs e transportam os aminoácidos correspondentes para formar a cadeia polipeptídica. Um peptídeo é composto por dois ou mais aminoáciodos, sendo classificados em dipeptídeos (2); tri; tetra; oligo e polipeptídeos; quando a cadeia de aminoácidos passa de 70, classifica-se em proteína.</li> 
<br></br>
<li> <b>Desdobramento e pós-tradução: </b> Após a síntese da cadeia polipeptídica, a proteína pode passar por um processo de dobramento ou enovelamento para adquirir sua estrutura tridimensional funcional. Após o dobramento, a proteína pode sofrer modificações pós-traducionais, como a adição de grupos químicos ou a clivagem de segmentos adicionais. </li>
</ol>
</p>
<br></br>
<p style = "font-size: 14px"> A imagem abaixo exemplifica o processo da tradução da fita de mRNA até a leitura pela fita de tRNA: </p>
<p style="text-align:center;"> <img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Aminoacids_table.svg/609px-Aminoacids_table.svg.png?20210405175054" alt = "Tabela de conversão de aminoácidos" width = "500" heigh = "500" align = center> </img> </p>
</div>



In [ ]:
''' Funções de conversão '''

# Função de DNA para mRNA
def dna_para_mRNA(sequencia_DNA):

    # Transcreve a sequência de DNA para mRNA através da substituição do T pelo U utilizando a função replace.

    return sequencia_DNA.replace('T', 'U')

# Função de mRNA para tRNA
def mRNA_para_tRNA(sequencia_mRNA):

    # Transcreve o mRNA para tRNA através da troca de cada nucleotídeo com seu respectivo par pelas regras de pareamento códon-anticódon
    
    dicionario_aux = {'A': 'U', 'U': 'A', 'C': 'G', 'G': 'C'}
    sequencia_tRNA = ""
    
    #Loop para iteração na variável
    for nucleotideo in sequencia_mRNA:
        if nucleotideo in dicionario_aux:
            sequencia_tRNA += dicionario_aux[nucleotideo]
        else:
            sequencia_tRNA += nucleotideo
    
    return sequencia_tRNA

# Função de mRNA e tRNA para aminoácidos
def traducao(sequencia_mRNA):

    # Traduz uma sequência de mRNA para a respectiva sequência de aminoácidos através do mapeamento da trinca de nucleotídeos (códons)  

    tabela_codon = {
        'AUG': 'M', 'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',  # ... Adicionar mais códons e seus respectivos aminoácidos
        'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S', 'UAU': 'Y',
        'UAC': 'Y', 'UAA': '*', 'UAG': '*', 'UGU': 'C', 'UGC': 'C',
        'UGA': '*', 'UGG': 'W', 'CUU': 'L', 'CUC': 'L', 'CUA': 'L',
        'CUG': 'L', 'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
        'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGU': 'R',
        'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AUU': 'I', 'AUC': 'I',
        'AUA': 'I', 'AUC': 'I', 'ACU': 'T', 'ACC': 'T', 'ACA': 'T',
        'ACG': 'T', 'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
        'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', 'GUU': 'V',
        'GUC': 'V', 'GUA': 'V', 'GUG': 'V', 'GCU': 'A', 'GCC': 'A',
        'GCA': 'A', 'GCG': 'A', 'GAU': 'D', 'GAC': 'D', 'GAA': 'E',
        'GAG': 'E', 'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
    }
    
    sequencia_proteina = ""
    
    # Loop de iteração da lista mRNA de input com a tabela de codons, forçando a leitura por codons
    for i in range(0, len(sequencia_mRNA), 3):
        codon = sequencia_mRNA[i:i+3]
        
        if codon in tabela_codon:
            amino_acido = tabela_codon[codon]
            if amino_acido == '*':  # Stop Codon encontrado
                break
            sequencia_proteina += amino_acido
    
    return sequencia_proteina

# confirmação que as funções foram declaradas
print("Funções OK.")

In [ ]:
# Simualação de uma replicação

sequencia_DNA = "ATGCATCGTAA"
sequencia_mRNA = dna_para_mRNA(sequencia_DNA)
sequencia_tRNA = mRNA_para_tRNA(sequencia_mRNA)
sequencia_proteina = traducao(sequencia_mRNA) # sequência de proteínas >= 70 Aminoácidos

print("Sequência de DNA:", sequencia_DNA)
print("Sequência de mRNA:", sequencia_mRNA)
print("Sequência de tRNA:", sequencia_tRNA)
print("Sequência de aminoácidos:", sequencia_proteina) 

<style>
  p {
    text-align: justify !important;
    font-size: 14px !important;
  }

  h3 {
  font-size: 24px
  }
</style>

<h3> <b> Machine Learning </b> </h3>

<div style = "font-size: 14px">
<p> De acordo com a literatura científica, uma proteína é composta por uma cadeia polipeptídica que pode conter mais de 70 aminoácidos. Como existem 20 tipos diferentes de aminoácidos que podem ser usados para construir uma proteína, e esses aminoácidos podem ser combinados em qualquer ordem, o número de sequências possíveis é virtualmente infinito. Portanto, as proteínas possuem combinações quase infinitas de aminoácidos. </p>

<p> Logo mm algoritmo de machine learning pode aprender a partir de dados experimentais como o DNA é transcrito em RNA e depois traduzido em proteínas, quais são os fatores que influenciam essa transformação e quais são as consequências de mutações ou alterações na expressão gênica.</p>

<p> Com o objetivo de facilitar o processo de tradução de uma sequência de DNA em uma proteína, o PreProt treina um modelo de aprendizado de máquina, e analisa os resultados, com uma base de dados contendo o proteoma da <i>Escherichia coli</i> K-12 substr. MG1655, que é a bactéria mais estudada na humanidade. </p>

</div>





In [ ]:
''' Carregamento e Tratamento dos dados'''

# Importando e tratando o dataset contendo o proteoma da E. Coli K-12 MG1655 em formato FASTA
from Bio import SeqIO
import pandas as pd

# Função para ler o arquivo FASTA e organizar os dados para criar um dataframe
def ler_fasta(file_path):
    sequences = [] # Criando uma lista vazia
    for record in SeqIO.parse(file_path, "fasta"): # iterando a base de dados na lista vazia
        sequences.append({"Sequencia_AA": str(record.seq), "ID_Proteina": record.id}) # criando um vetor bi-dimensional 

    return sequences

# Inputs
file_path = "datasets/Escherichia coli str. K-12 substr. MG1655/proteome.fasta"
sequencias = ler_fasta(file_path)

# Criando um Pandas Dataframe a partir do vetor sequences para posterior treinamento do modelo
df = pd.DataFrame(sequencias)
# Removendo a parte não utilizada do ID da proteína na coluna
df['ID_Proteina'] = df['ID_Proteina'].str.rsplit('|', n=1).str[-1]

print(df.head())
print(len(df.index)) #6463 proteínas

In [ ]:
# Analisando o proteoma da E. Coli K-12

# Importando biblioteca para leitura de arquivos FASTA
from Bio import SeqIO

# Abrindo o arquivo e apresentando algumas informações
sequences = [] # Criando uma lista vazia
for seq_record in SeqIO.parse("datasets/Escherichia coli str. K-12 substr. MG1655/proteome.fasta", "fasta"):
    # Adicionando o record na lista vazia
    sequences.append(str(seq_record.seq))
    # printando a sequência
    print(seq_record.seq)
    # printando o identificador da sequência (proteína)
    print(seq_record.id)
    # printando o tamanho da sequência (cadeia de aminoácidos)
    print(len(seq_record))

print("Sequências no arquivo:", len(sequences))

In [ ]:
# Código para o modelo de aprendizado de maquina - Classificador de Naive Bayes
# Escolha do modelo por ser um classificador de categorias

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Dividindo o Dataset em subsets de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df["Sequencia_AA"], df["ID_Proteina"], test_size=0.2, random_state=42)

In [ ]:
# PRIMEIRA VERSÃO
# Extração de características 
vectorizer = CountVectorizer()
X_train_encoded = vectorizer.fit_transform(X_train)
X_test_encoded = vectorizer.transform(X_test)

In [ ]:
# SEGUNDA VERSÃO
# Trocando o Encoding para melhorar acurácia: k-mer length
from sklearn.feature_extraction.text import CountVectorizer

# Step 2: Convert Protein Sequences to Numerical Features (k-mer frequency encoding)
k = 1  # Specify the length of k-mers

# Convert protein sequences to k-mer frequency representation
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k, k))
X_train_encoded = vectorizer.fit_transform(X_train)
X_test_encoded = vectorizer.transform(X_test)

In [ ]:
# TERCEIRA VERSÃO
# Multiple sequence alignment (MSA) - O método utilizado para o caso de sequência de proteínas em machine learning
# primeiramente realizando um PSI-BLAST https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROGRAM=blastp&BLAST_PROGRAMS=psiBlast com o proteoma como input
# O PSI-BLAST tem como objetivo encontrar proteínas com sequências e funções parecidas, gerando uma position-specific scoring matrices (PSSMs)
# Então essa PSSMs pode ser utilizada para treinar nosso algorítimo de classificação
# Para iterar o PSI-BLAST para cada proteína do proteoma da E. Coli K-12 necessitaria de MUITO poder computacional, logo vou baixar uma versão de PSSM do NCBI
# O uso desse modelo nos permite chegar a sequencia, estrutura e função da proteína!! https://www.ncbi.nlm.nih.gov/Structure/cdd/cdd.shtml
# Input do proteoma aqui https://www.ncbi.nlm.nih.gov/Structure/cdd/cdd_help.shtml#BatchRPSBInput
# Checar arquivo pssm.py

In [ ]:
# Treinando o modelo
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_encoded, y_train)

In [ ]:
# Aplicando métricas de análise do modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_train = naive_bayes_model.predict(X_train_encoded)
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train, average='weighted')
train_recall = recall_score(y_train, y_pred_train, average='weighted')
train_f1 = f1_score(y_train, y_pred_train, average='weighted')

print("Acurácia:", train_accuracy)
print("Precisão:", train_precision)
print("Recall:", train_recall)
print("F1-Score:", train_f1)

In [ ]:
# Criando gráficos e visualizações do modelo
import matplotlib.pyplot as plt
import numpy as np

metricas = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
valores = [train_accuracy, train_precision, train_recall, train_f1]
valores_percentuais = [value * 100 for value in values]

plt.figure(figsize=(8, 4))
plt.bar(metricas, valores_percentuais, color='steelblue')
plt.xlabel('Métricas')
plt.ylabel('Valores')
plt.title('Avaliação do Modelo - Métricas')
plt.ylim([0, 1])

# Set y-axis tick positions and labels as percentages
plt.yticks(np.linspace(0, 100, 11), ['{}%'.format(int(valores_percentuais)) for valores_percentuais in np.linspace(0, 100, 11)])

plt.show()

In [ ]:
# Predições
nova_sequencia = ["MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVLTSVKKAEQYLLENETTKNYLGIDGIPEFGRCTQELLFGKGSALINDKRARTAQTPGGTGALRVAADFLAKNTSVKRVWVSNPSWPNHKSVFNSAGLEVREYAYYDAENHTLDFDALINSLNEAQAGDVVLFHGCCHNPTGIDPTLEQWQTLAQLSVEKGWLPLFDFAYQGFARGLEEDAEGLRAFAAMHKELIVASSYSKNFGLYNERVGACTLVAADSETVDRAFSQMKAAIRANYSNPPAHGASVVATILSNDALRAIWEQELTDMRQRIQRMRQLFVNTLQEKGANRDFSFIIKQNGMFSFSGLTKEQVLRLREEFGVYAVSGRVNVAGMTPDNMAPLCEAIVAVL"] # NOVO INPUT
# essa é a sequência de aminoácidos da proteína I

# INSA9_ECOLI
# MASVSISCPSCSATDGVVRNGKSTAGHQRYLCSHCRKTWQLQFTYTASQPGTHQKIIDMAMNGVGCRATARIMGVGLNTILRHLKNSGRSR

# AAT_ECOLI
# MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVLTSVKKAEQYLLENETTKNYLGIDGIPEFGRCTQELLFGKGSALINDKRARTAQTPGGTGALRVAADFLAKNTSVKRVWVSNPSWPNHKSVFNSAGLEVREYAYYDAENHTLDFDALINSLNEAQAGDVVLFHGCCHNPTGIDPTLEQWQTLAQLSVEKGWLPLFDFAYQGFARGLEEDAEGLRAFAAMHKELIVASSYSKNFGLYNERVGACTLVAADSETVDRAFSQMKAAIRANYSNPPAHGASVVATILSNDALRAIWEQELTDMRQRIQRMRQLFVNTLQEKGANRDFSFIIKQNGMFSFSGLTKEQVLRLREEFGVYAVSGRVNVAGMTPDNMAPLCEAIVAVL

# Se for uma sequência de DNA, usar as funções de tradução: fazer um if no algorítimo

sequencia_mRNA = dna_para_mRNA(sequencia_DNA)
sequencia_tRNA = mRNA_para_tRNA(sequencia_mRNA)
sequencia_proteina = traducao(sequencia_mRNA)

nova_sequencia_encoded = vectorizer.transform(nova_sequencia)
nome_proteina = naive_bayes_model.predict(nova_sequencia_encoded)
print("Nome da possível proteína:", nome_proteina)

In [7]:
# tratamento do xlsx chaperone interactors
import pandas as pd
df = pd.read_csv("datasets/MG1655_Chaperone_Interactors.csv")


,Accession_STEPdb_2.0,Protein_ID_STEPdb_2.0,1ary_Gene_Name,STEPdb_Universal_Name,Uniprot_Protein_Name,STEPdb_Sub-cellular_Location_Full Name,STEPdb_Sub-cellular_Location_Letter Code,sub-cellular_topology_group,SRP_interactome_Schibich2016,TF_interactome_Oh2011,...,TF_client_Martinez2009,TF_client_Butland2005,TF_client_Arifuzzaman2006,TF_client_Niwa2009,GroEL_client_Kerner2005,GroEL_client_Arifuzzaman2006,GroEL_client_Chapman2006,GroEL_client_Niwa2009,SecB_client_Stepdb1.0,SecB_client_additional_data
0,P0DPC5,YTID_ECOLI,ytiD,"cytoplasmic protein, YtiD",Protein YtiD,Cytoplasmic,A,Cytoplasmic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P0DPC4,YTIC_ECOLI,ytiC,"cytoplasmic protein, YtiC",Protein YtiC,Cytoplasmic,A,Cytoplasmic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P0DN74,YTIA_ECOLI,ytiA,"cytoplasmic protein, YtiA",Uncharacterized protein YtiA,Cytoplasmic,A,Cytoplasmic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V9HVX0,YPAA_ECOLI,ypaA,"cytoplasmic protein, YpaA",Uncharacterized protein YpaA,Cytoplasmic,A,Cytoplasmic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P0DPC9,YNFQ_ECOLI,ynfQ,"cytoplasmic protein, YnfQ",Protein YnfQ,Cytoplasmic,A,Cytoplasmic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
